In [48]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [49]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [50]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [51]:
# Extract tracks from API
output_df = pd.DataFrame()
for t in tqdm(results[340001:350001]):
      output = sp.audio_features(t)
      output_df = output_df.append(pd.DataFrame(output))
output_df.head()

100%|██████████| 10000/10000 [16:26<00:00, 10.14it/s]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.612,0.117,9,-15.357,1,0.1770,0.9900,0.000310,0.0788,0.781,172.444,audio_features,1a8JNXclpuzv9Rx8KxBbIO,spotify:track:1a8JNXclpuzv9Rx8KxBbIO,https://api.spotify.com/v1/tracks/1a8JNXclpuzv...,https://api.spotify.com/v1/audio-analysis/1a8J...,153260,4
0,0.716,0.101,0,-13.472,1,0.0881,0.9720,0.871000,0.1180,0.336,85.006,audio_features,1a8jOpvRqrBIBPptN0LZnm,spotify:track:1a8jOpvRqrBIBPptN0LZnm,https://api.spotify.com/v1/tracks/1a8jOpvRqrBI...,https://api.spotify.com/v1/audio-analysis/1a8j...,139549,3
0,0.483,0.366,3,-8.198,1,0.0297,0.7890,0.000000,0.2080,0.264,124.081,audio_features,1a8joZoDooqsCTZBTF14RP,spotify:track:1a8joZoDooqsCTZBTF14RP,https://api.spotify.com/v1/tracks/1a8joZoDooqs...,https://api.spotify.com/v1/audio-analysis/1a8j...,236800,4
0,0.829,0.463,6,-9.878,1,0.0313,0.6970,0.000013,0.1170,0.504,111.963,audio_features,1a8JZlyrP1UdX9nEZdLzxX,spotify:track:1a8JZlyrP1UdX9nEZdLzxX,https://api.spotify.com/v1/tracks/1a8JZlyrP1Ud...,https://api.spotify.com/v1/audio-analysis/1a8J...,236470,4
0,0.338,0.567,9,-8.826,1,0.1370,0.0904,0.000000,0.0669,0.297,97.551,audio_features,1A8k3E7mFZm0Oj7tAt86gK,spotify:track:1A8k3E7mFZm0Oj7tAt86gK,https://api.spotify.com/v1/tracks/1A8k3E7mFZm0...,https://api.spotify.com/v1/audio-analysis/1A8k...,262493,4


In [52]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

10000 rows imported to database.
App contains (350001,) songs.
